In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import geopandas as gpd
import shapely

In [ ]:
brazil_gdf = gpd.read_file('/home/ksolvik/research/reservoirs/analysis/data/misc/general_borders/Brazilian_States_aea.shp')


In [ ]:
brazil_gdf = gpd.read_file('/home/ksolvik/research/reservoirs/analysis/data/misc/general_borders/Brazilian_States_aea.shp')
brazil_gdf = brazil_gdf.to_crs('EPSG:4326')

In [ ]:
brazil_gdf = brazil_gdf.dissolve(by='REGIAO')

In [ ]:
all_csvs = glob.glob('./out/wgs84/ls*merged.csv')
all_csvs.sort()

In [ ]:
# # Temporary: Convert to lat/lon
# for csv in all_csvs:
#     out_path = csv.replace('aea', 'wgs84')
#     temp_df = pd.read_csv(csv)
#     temp_df = temp_df[['center_lat', 'center_lon', 'hydropoly_max', 'area']]
#     gdf = gpd.GeoDataFrame(
#         temp_df, geometry=gpd.points_from_xy(temp_df.center_lon, temp_df.center_lat),
#         crs='ESRI:102033'
#     )
#     gdf_wgs84 = gdf.to_crs('EPSG:4326')
#     gdf_wgs84['longitude'] = gdf_wgs84.geometry.x
#     gdf_wgs84['latitude'] = gdf_wgs84.geometry.y
#     gdf_wgs84.drop(columns=['geometry']).to_csv(out_path, index=False)

In [ ]:
def read_process_csv(csv):
    temp_df = pd.read_csv(csv)
    temp_df['satellite'] = os.path.basename(csv)[:3]
    temp_df['year'] = int(os.path.basename(csv)[4:8])
    return temp_df

In [ ]:
full_df = pd.concat([
    read_process_csv(csv) for csv in all_csvs
])

In [ ]:
full_df = full_df.loc[full_df['hydropoly_max']<100]

In [ ]:
full_df['area'] = full_df['area']*100/10000
full_df = full_df.loc[full_df['area']<100]

In [ ]:

# xlims = brazil_gdf.bounds.min()['minx'], brazil_gdf.bounds.max()['maxx']
# xlims = (xlims[0] - 1.5, xlims[1]-3)

# ylims = brazil_gdf.bounds.min()['miny'], brazil_gdf.bounds.max()['maxy']
# ylims = (ylims[0] - 1, ylims[1] + 1)
# ylims_range = ylims[1] - ylims[0]
# xlims_range = xlims[1] - xlims[0]

In [ ]:
xlims = brazil_gdf.bounds.min()['minx'], brazil_gdf.bounds.max()['maxx']
xlims = (xlims[0] - 1.5, xlims[1]-4)
ylims = brazil_gdf.bounds.min()['miny'], brazil_gdf.bounds.max()['maxy']
ylims = (ylims[0] - 1, ylims[1] + 1)
ylims_range = ylims[1] - ylims[0]
xlims_range = xlims[1] - xlims[0]

In [ ]:
# Plot area in degrees lat
xlims_range*ylims_range

In [ ]:
%matplotlib inline

axes_height_ratios=[1, 0.05]
fig, axs = plt.subplots(2, 2, figsize = (24, 16),
                       gridspec_kw={"height_ratios":axes_height_ratios})

# Plot 1984 first
brazil_gdf.boundary.plot(ax=axs[0, 0], color='red', alpha=0.3)


outline_gdf = gpd.GeoDataFrame(
    geometry=gpd.GeoSeries(shapely.geometry.Polygon(
        [[xlims[0], ylims[0]],
         [xlims[0], ylims[1]],
         [xlims[1], ylims[1]],
         [xlims[1], ylims[0]],
         [xlims[0], ylims[0]]])),
    crs='EPSG:4326')
nonbrazil_poly = outline_gdf.overlay(brazil_gdf, how='difference')


year_1984_df = full_df.loc[full_df['year']==1984]
axs[0,0].hexbin(year_1984_df['longitude'], year_1984_df['latitude'],
              gridsize=(60, int(60*ylims_range/xlims_range)),
             vmin=0, vmax=3000,
             extent=xlims + ylims)

# Plot 2022
brazil_gdf.boundary.plot(ax=axs[0, 1], color='red', alpha=0.3)
year_2022_df = full_df.loc[full_df['year']==2022]
im = axs[0, 1].hexbin(year_2022_df['longitude'], year_2022_df['latitude'],
              gridsize=(60, int(60*ylims_range/xlims_range)),
             vmin=0, vmax=3000,
             extent=xlims + ylims)

# fig.colorbar(im)

for cur_ax in axs[0]:
    cur_ax.set_xlabel('Longitude (deg)')
    cur_ax.set_ylabel('Latitude (deg)')
    cur_ax.set_xlim(xlims)
    cur_ax.set_ylim(ylims)
axs[0,0].set_title('1984', size=20)
axs[0,1].set_title('2022', size=20)

# Remove outside of Brazil
nonbrazil_poly.plot(ax=axs[0,1],color='white')
nonbrazil_poly.plot(ax=axs[0,0],color='white')

# Set up colorbar
gs = axs[0, 0].get_gridspec()
for ax in axs[-1]:
    ax.remove()
axbig = fig.add_subplot(gs[-1, :])
axbig.set_title('Reservoir Count')
fig.colorbar(im, cax=axbig, orientation='horizontal')
fig.tight_layout()

In [ ]:
# %matplotlib notebook

# fig, ax = plt.subplots(figsize=(8,16))
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)

# year_start = 1985
# year_end = 2021

# def animate(i):
#     df = full_df.loc[full_df['year']==i]
#     x = df['center_lon'].values
#     y = df['center_lat'].values
#     if i != year_start:
#         ax.collections[-1].remove()
#     hexlayer = ax.hexbin(x,y, gridsize=grid_size,
#                          alpha=1.0,
#                         vmin=0, vmax=1000,
#                         extent=(xlims[0], xlims[1], ylims[0], ylims[1]))
#     ax.set_title(i)
#     return hexlayer,

# ani = FuncAnimation(fig, animate, frames=np.arange(year_start, year_end, 5), interval=500)
# ani.save('test_animation_nols7.gif')